# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [13]:
from google.colab import drive

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Submission Dicoding/hasil_clustering_frauds_detection.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df.head()

,cat__TransactionType_Credit,cat__TransactionType_Debit,cat__Channel_ATM,cat__Channel_Branch,cat__Channel_Online,cat__CustomerOccupation_Engineer,cat__CustomerOccupation_Retired,cat__CustomerOccupation_Student,remainder__TransactionAmount,remainder__CustomerAge,remainder__TransactionDuration,remainder__LoginAttempts,remainder__AccountBalance,Cluster_DBSCAN,Cluster_Description
0,-0.543467,0.543467,1.425787,-0.730676,-0.689492,-0.576869,-0.559537,-0.593558,-1.112175,1.424991,-0.550392,-0.207599,-0.007141,-1,Fraud (Outliers)
1,-0.543467,0.543467,1.425787,-0.730676,-0.689492,-0.576869,-0.559537,-0.593558,0.558073,1.312326,0.306226,-0.207599,2.209057,-1,Fraud (Outliers)
2,-0.543467,0.543467,-0.701367,-0.730676,1.450342,-0.576869,-0.559537,1.684755,-0.594705,-1.447980,-0.907317,-0.207599,-1.029765,-1,Fraud (Outliers)
3,-0.543467,0.543467,-0.701367,-0.730676,1.450342,-0.576869,-0.559537,1.684755,-0.326238,-1.053651,-1.349903,-0.207599,0.878869,-1,Fraud (Outliers)
4,1.840038,-1.840038,-0.701367,-0.730676,1.450342,-0.576869,-0.559537,1.684755,-1.115127,-1.053651,1.120013,-0.207599,0.586767,-1,Fraud (Outliers)


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [15]:
df['Label_Fraud'] = df['Cluster_Description'].map({'Fraud(Outliers)': 1, 'Normal': 0})
X = df.drop(columns=['Cluster_DBSCAN', 'Cluster_Description'])
y = df['Cluster_Description']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42, stratify = y)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
# Model Random Forest
model = RandomForestClassifier(
    n_estimators=100,       #jumlah pohon yang diuji coba dengan subsample ada 100
    max_depth=10,           #jumlah kedalaman tiap pohon ada 10, ini hyperparameter yang umum agar tidak overfit
    min_samples_split=2,    #ukuran split tiap node dalam pohon sebanyak 2 kali
    min_samples_leaf=1,     #ukuran minimum leaf node
    random_state=42,        #langkah umum untuk memastikan hasil konsisten
    max_features = 'sqrt',  #fitur yang digunakan dalam node-node tiap pohon dibatasi sebanyak 'sqrt' dari total fitur
    bootstrap =True         #memastikan tiap pohon mendapat subsample yang berbeda
)

model.fit(X_train, y_train)  #melatih data fitur x (independent variabel) dan y (dependent variabel)
y_pred = model.predict(X_test)  #prediksi data fitur x

Membuat model klasifikasi data transaksi fraud atau tidak fraud (normal) dengan Random Forest. model Random Forest dibuat langsung dengan beberapa hyperparameter.

In [20]:
# Model Support Vector Machine
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.01, 0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__degree': [2, 3, 4]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(f'Model terbaik: {best_params}')

Model terbaik: {'svm__C': 100, 'svm__degree': 2, 'svm__gamma': 'auto', 'svm__kernel': 'rbf'}


Membuat model klasifikasi data transaksi fraud atau tidak fraud (normal) dengan Support Vector Machine (SVM). Tidak seperti model Random Forest yang langsung membuat dugaan hyperparameter awal, model SVM terlebih dahulu langsung di tuning untuk mendapat hyperparameter terbaik.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [6]:
evaluasi = classification_report(y_test, y_pred)
print(evaluasi)

                  precision    recall  f1-score   support

Fraud (Outliers)       0.98      1.00      0.99       444
          Normal       1.00      0.81      0.89        36

        accuracy                           0.99       480
       macro avg       0.99      0.90      0.94       480
    weighted avg       0.99      0.99      0.98       480



Hasil evaluasi:
*   Presisi: Untuk fraud, model secara keseluruhan memprediksi bahwa 98% data transaksi adalah fraud, sementara untuk yang normal, model secara keseluruhan tepat dalam memprediksi 100% data tersebut adalah normal.
*   Recall: Pada fraud, model 100% dapat mengenali fraud (false negative nya 0), sementara pada normal, model hanya dapat mengenali 81% secara keseluruahan, sisanya mungkin dianggap fraud.

*   F1-score: Pada fraud mendapat hasil sangat baik yaitu 99% sedangkan normal mendapat hasil 89%
*   Support: Jumlah data test set pada fraud sebanyak 444 dan normal sebanyak 36 data.

Secara keseluruhan hasil evaluasi cukup baik, namun untuk recall masih dalam kategori "kurang" ditambah minimum f1-score yang harus diperoleh yaitu 91%.





In [23]:
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

                  precision    recall  f1-score   support

Fraud (Outliers)       1.00      1.00      1.00       444
          Normal       0.94      0.94      0.94        36

        accuracy                           0.99       480
       macro avg       0.97      0.97      0.97       480
    weighted avg       0.99      0.99      0.99       480



Hasil evaluasi:

*   Presisi: Untuk fraud, model secara keseluruhan dengan tepat memprediksi data fraud 100%, sementara data normal dapat diprediksi model dengan presisi 94%

*   Recall: Pada fraud, model 100% dapat mengenali fraud (false negative nya 0), sementara pada normal, model dapat mengenali 94% secara keseluruahan, sisanya mungkin dianggap fraud.

*   F1-score: Pada fraud mendapat hasil sempurna yaitu 100%, sedangkan normal mendapat hasil 94%

*   Support: Jumlah data test set pada fraud sebanyak 444 dan normal sebanyak 36 data.

Secara keseluruhan Model SVM yang telah dioptimasi dengan GridSearchCV
mampu memisahkan outlier dari data normal dengan akurasi tinggi dan presisi tinggi. Sangat cocok untuk digunakan dalam skenario deteksi fraud, terutama ketika penting untuk tidak salah mendeteksi data fraud maupun normal.

## **c. Tuning Model Klasifikasi (Random Forest)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [7]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

Best Parameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}


Membuat hyperparameter tuning untuk meningkatkan performa model terkhususnya pada bagian recall.

In [8]:
model = RandomForestClassifier(
    n_estimators=50,
    max_depth=None,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    max_features = 'sqrt',
    bootstrap =False
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Random Forest)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [9]:
evaluasi = classification_report(y_test, y_pred)
print(evaluasi)

                  precision    recall  f1-score   support

Fraud (Outliers)       0.99      1.00      0.99       444
          Normal       1.00      0.83      0.91        36

        accuracy                           0.99       480
       macro avg       0.99      0.92      0.95       480
    weighted avg       0.99      0.99      0.99       480



Hasil evaluasi setelah melakukan tuning hyperparameter dengan GridSearchCV menjadi lebih baik, dengan meningkatnya performa presisi fraud, recall pada data normal, dan juga f1-score pada data normal